## Segmentation notebook

In this notebook we explore some of the segmentation tools the `qim3d` library provides.

This example contains an image of air bubbles in a 3d volume representing a cutout of cement. The goal of the example is to detect the amount of holes and visualize the largest of them.

First we load the image from the `qim3d.examples` module:

In [1]:
import qim3d
vol = qim3d.examples.cement_128x128x128
qim3d.viz.slicer(vol)

interactive(children=(IntSlider(value=64, description='Slice', max=127), Output()), _dom_classes=('widget-inte…

Next we binarize the image by applying a threshold. We visualize once again to confirm the threshold is not too high or low:

In [6]:
vol_t = vol < 60

qim3d.viz.slicer(vol_t)

interactive(children=(IntSlider(value=64, description='Slice', max=127), Output()), _dom_classes=('widget-inte…

Now the image can be segmented using the `get_3d_cc` method. This method calculates the connected components of a volume and returns a custom CC class. This class contains the `get_cc` method, that returns the segmented volume.

In [3]:
cc = qim3d.segmentation.get_3d_cc(vol_t)
vol_cc = cc.get_cc()

Total number of connected components found: 1845


We see that 1845 connectec components were wound. However many of these are very small, and we would only like to find the largest ones. Therefore, we create a for-loop over the connected components, and remove any that are smaller than 300 voxels:

In [4]:
for i in range(1, len(cc)):
    if (vol_cc==i).sum() < 300:
        vol_cc[vol_cc == i] = 0

vol_cc = vol_cc > 100

Now we can visualize it once again to confirm our operations have worked as intended:

In [5]:
qim3d.viz.slicer(vol_cc)

interactive(children=(IntSlider(value=64, description='Slice', max=127), Output()), _dom_classes=('widget-inte…

In order to enable better visual distinction between the airbubbles, we can use the `watershed` method:

In [7]:
vol_label, num_labels = qim3d.segmentation.watershed(vol_cc, min_distance=5)

Total number of objects found: 210


To make different color for every airbubble, we create our own colormap using the `viz.colormaps.segmentation` method:

In [8]:
color_map = qim3d.viz.colormaps.segmentation(num_labels)
qim3d.viz.slicer(vol_label, color_map=color_map)

interactive(children=(IntSlider(value=64, description='Slice', max=127), Output()), _dom_classes=('widget-inte…